In [ ]:
# 練習爬取meta標籤
import requests,bs4

headers={'user-agent':'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
url='https://www.bqkan8.com/50_50096/'
html=requests.get(url,headers=headers)
# 轉gbk是因為這個網頁的編碼是gbk,要先檢查網頁
html.encoding='gbk'   # 要在轉bs4前變更編碼
obj_soup=bs4.BeautifulSoup(html.text,'lxml')

print('編碼:',html.encoding)
#print(obj_soup)

# 爬到meta的property 就可以直接印出content了
book_title=obj_soup.find('meta',property='og:title')
book_author=obj_soup.find('meta',property='og:novel:author')
book_description=obj_soup.find('meta',property='og:description')

print('作者:',book_title['content'])
print('書名:',book_author['content'])
print('內文:',book_description['content'].strip())

In [ ]:
# 自己練習別的小說網站 因為按搜尋跑出這網址 把網頁內的文章都存成txt
import requests,bs4,os

headers={'user-agent':'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
url='https://www.qu-la.com/booktxt/37517808116/'
html=requests.get(url,headers=headers)
# 轉gbk是因為這個網頁的編碼是gbk,要先檢查網頁
html.encoding='gbk'   # 要在轉bs4前變更編碼
obj_soup=bs4.BeautifulSoup(html.text,'lxml')

txt_dir='out_14_me'
if os.path.exists(txt_dir)==False:
    os.mkdir(txt_dir)

obj_title=obj_soup.find('div','book-chapter-list').find_all('ul')[1].find_all('li')
#print(obj_title)
for t in obj_title:
    book_title=t.text+'.txt'
    book_url='https://www.qu-la.com'+t.a['href']
    print(book_title)
    print(book_url)
    print()
    
    # 抓取點入標題網址的內文
    content_html=requests.get(book_url)
    content_html.encoding='gbk'
    content_soup=bs4.BeautifulSoup(content_html.text,'lxml')
    
    # 找到內文區塊全部抓下
    content_txt=content_soup.find('div','chapter-box').text.strip()
    print(content_txt)
    txt_file=open(os.path.join(txt_dir,book_title),'w',newline='',encoding='utf-8')
    try:
        txt_file.write(content_txt)
        print('儲存成功',book_title,'\n')
    except Exception:
        print('儲存失敗')
txt_file.close()

In [ ]:
# 自己練習課本小說網站 把網頁內的文章都存成txt
import requests,bs4,os

url='https://www.bqkan8.com/50_50096/'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
html=requests.get(url)
html.encoding='gbk'
obj_soup=bs4.BeautifulSoup(html.text,'lxml')

txt_dir='out_14_1_me'
if os.path.exists(txt_dir)==False:
    os.mkdir(txt_dir)

# 要移除的多餘內容 
remove_list=['&1t;/p>','&nbsp;',' ','请记住本书首发域名：www.bqkan8.com。笔趣阁手机版阅读网址：m.bqkan8.com']

obj_div=obj_soup.find('div','listmain').dl.find_all('dd') 
for i,obj in enumerate(obj_div[12:]):
    #print(i)  # 用索引方式得知前12回不要 所以從第12開始[12:]
    book_title=obj.text.strip()+'.txt'
    book_url='https://www.bqkan8.com'+obj.a['href']
    remove_='('+book_url+')'
    remove_list.append(remove_)
    #print(book_title)
    print(book_url)
    
    # 進入文章網頁抓內文
    content_html=requests.get(book_url)
    content_html.encoding='gbk'
    content_soup=bs4.BeautifulSoup(content_html.text,'lxml')
    
    content_h1=content_soup.find('h1').text.strip()+'\n'
    content_txt=content_soup.find('div',id='content').text.strip()
    
    for r in remove_list:     
        content_txt=content_txt.replace(r,'')
    
#     print(content_h1)
#     print(content_txt)
    
    txt_file=open(os.path.join(txt_dir,book_title),'w',newline='',encoding='utf-8')
    try:
        txt_file.write(content_h1)
        txt_file.write(content_txt)
        print('儲存成功',book_title)
        print(i)
    except Exception:
        print('儲存失敗')
print('全部下載完成')
txt_file.close()

In [ ]:
# 自己練習課本小說網站 把網頁內的文章都存成txt 直接進單篇
import requests,bs4,os

url='https://www.bqkan8.com/50_50096/18520426.html'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
html=requests.get(url)
html.encoding='gbk'
obj_soup=bs4.BeautifulSoup(html.text,'lxml')

# 要移除的多餘內容 
remove_list=['&1t;/p>','&nbsp;',' ','请记住本书首发域名：www.bqkan8.com。笔趣阁手机版阅读网址：m.bqkan8.com']

story=''
obj_title=obj_soup.find('div','content').h1.text.strip()
obj_content=obj_soup.find('div',id='content')
story=story+obj_title+'\n'  # 先把標題加入
remove_='('+url+')'
remove_list.append(remove_)

# 一段一段檢查
for content in obj_content:
#     print(len(content))
#     print(type(content))
    # 不是bs4.element.NavigableString型態 表示內文是空的
    if type(content)==bs4.element.NavigableString:
        txt_content=content.text.strip()
        if type(txt_content)==str and txt_content!='':
            txt_content=content.text.strip()
            for r in remove_list:
                txt_content=txt_content.replace(r,'')
            story=story+txt_content+'\n\n'          # 依序把有字串非空值的小說內文加入 加\n使內文更整齊
#     else:
#         print('NO') 
print(story)
fn=obj_title+'.txt'
with open(fn,'w',encoding='utf-8') as file:
    file.write(story)

In [5]:
# 自己練習課本小說網站 把網頁內的文章都存成txt 從外進去全部
import requests,bs4,os

url='https://www.bqkan8.com/50_50096/'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
html=requests.get(url)
html.encoding='gbk'
obj_soup=bs4.BeautifulSoup(html.text,'lxml')

txt_dir='out_14_2_me'
if os.path.exists(txt_dir)==False:
    os.mkdir(txt_dir)

# 要移除的多餘內容 
remove_list=['&1t;/p>','&nbsp;',' ','请记住本书首发域名：www.bqkan8.com。笔趣阁手机版阅读网址：m.bqkan8.com']

obj_div=obj_soup.find('div','listmain').dl.find_all('dd') 
for i,obj in enumerate(obj_div[12:]):
    story='' # 變空值存下一回
    #print(i)  # 用索引方式得知前12回不要 所以從第12開始[12:]
    book_title=obj.text.strip()+'.txt'
    book_url='https://www.bqkan8.com'+obj.a['href']
    remove_='('+book_url+')'
    remove_list.append(remove_)
    
    # 進入文章網頁抓內文
    content_html=requests.get(book_url)
    content_html.encoding='gbk'
    content_soup=bs4.BeautifulSoup(content_html.text,'lxml')
    
    content_h1=content_soup.find('h1').text.strip()+'\n'
    story=story+content_h1+'\n'
    content_txt=content_soup.find('div',id='content')  # 因為要檢查 所以後面不可加不可加text.strip() 
    # 一段一段檢查 要檢查的內文要是完整的html 不可加text.strip() 
    for content in content_txt:
        if type(content)==bs4.element.NavigableString:
            txt_content=content.text.strip()  # 這裡就一定要加text.strip() 不然下面無法判斷type=str
            #print(txt_content)
            if type(txt_content)==str and txt_content!='':
                txt_content=content
                for r in remove_list:
                    txt_content=txt_content.replace(r,'')
                    #print(txt_content)
                story=story+txt_content+'\n\n'
#         else:
#             print('NO')
    #print(story)
    txt_file=open(os.path.join(txt_dir,book_title),'w',newline='',encoding='utf-8')
    try:
        txt_file.write(story)
        print('儲存成功',book_title)
        print(i)
    except Exception:
        print('儲存失敗')
print('全部儲存完成')
txt_file.close()

儲存成功 第一回 宴桃园豪杰三结义 斩黄巾英雄首立功.txt
0
儲存成功 第二回 张翼德怒鞭督邮 何国舅谋诛宦竖.txt
1
儲存成功 第三回 议温明董卓叱丁原 馈金珠李肃说吕布.txt
2
儲存成功 第四回 废汉帝陈留践位 谋董贼孟德献刀.txt
3
儲存成功 第五回 发矫诏诸镇应曹公 破关兵三英战吕布.txt
4
儲存成功 第六回 焚金阙董卓行凶 匿玉玺孙坚背约.txt
5
儲存成功 第七回 袁绍磐河战公孙 孙坚跨江击刘表.txt
6
儲存成功 第八回 王司徒巧使连环计 董太师大闹凤仪亭.txt
7
儲存成功 第九回 除暴凶吕布助司徒 犯长安李倔听贾诩.txt
8
儲存成功 第十回 勤王室马腾举义 报父仇曹操兴师.txt
9
儲存成功 第十一回 刘皇叔北海救孔融 吕温侯濮阳破曹操.txt
10
儲存成功 第十二回 陶恭祖三让徐州 曹孟穗大战吕布.txt
11
儲存成功 第十三回 李傕郭汜大交兵 杨奉董承双救驾.txt
12
儲存成功 第十四回 曹孟德移驾幸许都 吕奉先乘夜袭徐郡.txt
13
儲存成功 第十五回 太史慈酣斗小霸王 孙伯符大战严白虎.txt
14
儲存成功 第十六回 吕奉先射戟辕门 曹孟德败师淯水.txt
15
儲存成功 第十七回 袁公路大起七军 曹孟德会合三将.txt
16
儲存成功 第十八回 贾文和料敌决胜 夏侯惇拨矢啖睛.txt
17
儲存成功 第十九回 下邳城曹操鏖兵 白门楼吕布殒命.txt
18
儲存成功 第二十回 曹阿瞒许田打围 董国舅内阁受诏.txt
19
儲存成功 第二十一回 曹操煮酒论英雄 关公赚城斩车胄.txt
20
儲存成功 第二十二回 袁曹各起马步三军 关张共擒王刘二将.txt
21
儲存成功 第二十三回 祢正平裸衣骂贼 吉太医下毒遭刑.txt
22
儲存成功 第二十四回 国贼行凶杀贵妃 皇叔败走投袁绍.txt
23
儲存成功 第二十五回 屯土山关公约三事 救白马曹操解重围.txt
24
儲存成功 第二十六回 袁本初败兵折将 关云长挂印封金.txt
25
儲存成功 第二十七回 美髯公千里走单骑 汉寿侯五关斩六将.txt
26
儲存成功 第二十八回 斩蔡阳兄弟释疑 会古城主臣聚义.txt
27
儲存成功 第二十九回 小霸王怒斩于吉 碧眼儿坐领江东.txt
28
儲存成功 第三十回 战官渡本初败绩 劫乌巢孟德烧粮.txt
29

In [1]:
content='我是誰哈哈 !!\n@@ ## ()'
oo=['是','!!',' ','\n']
for o in oo:
    content=content.replace(o,'')
print(content)

我誰哈哈@@##()
